#### The main aim of this project is to predict which patient could die due to heart attack 

#### I am new on Kaggle and this is my first notebook so if you like it then dont forget to Upvote and Comment down if         you have any Suggestion

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#loading data
df = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
df.head()


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['DEATH_EVENT'].unique()

In [ ]:
df.isnull().sum()

 * Sex - Gender of patient Male = 1, Female =0
 * Diabetes - 0 = No, 1 = Yes
 * Anaemia - 0 = No, 1 = Yes
 * High_blood_pressure - 0 = No, 1 = Yes
 * Smoking - 0 = No, 1 = Yes
 * DEATH_EVENT - 0 = No, 1 = Yes

In [ ]:
df['anaemia'].unique()

In [ ]:
df.groupby(['anaemia'])['DEATH_EVENT'].mean()

# EDA of Categorical and Dependent Variable

In [ ]:
import seaborn as sns

In [ ]:
g = sns.factorplot(x="DEATH_EVENT",hue='sex',data=df,kind="count", size = 6 , 
palette = "muted")
g = g.set_ylabels("Count of death between male and female")

In [ ]:
g = sns.factorplot(x="anaemia",y="DEATH_EVENT",hue='sex',data=df,kind="bar", size = 6 , 
palette = "muted")
g = g.set_ylabels("Death Events probability")

In [ ]:
g = sns.factorplot(x="diabetes",y="DEATH_EVENT",hue='sex',data=df,kind="bar", size = 6 , 
palette = "muted")
g = g.set_ylabels("Death Events probability")

In [ ]:
g = sns.factorplot(x="high_blood_pressure",y="DEATH_EVENT",hue='sex',data=df,kind="bar", size = 6 , 
palette = "muted")
g = g.set_ylabels("Death Events probability")

In [ ]:
g = sns.factorplot(x="smoking",y="DEATH_EVENT",hue='sex',data=df,kind="bar", size = 6 , 
palette = "muted")
g = g.set_ylabels("Death Events probability")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1

for column in df:
    if plotnumber<=14 :     # as there are 9 columns in the data
        ax = plt.subplot(5,3,plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column,fontsize=20)
        #plt.ylabel('Salary',fontsize=20)
    plotnumber+=1
plt.show()

In [ ]:
import scipy.stats as stats

In [ ]:
def check_normality(data, name):
    shap_t,shap_p = stats.shapiro(data)
    print(name + ' parameters:')
    print()
    print("Skewness: %f" % abs(data).skew())
    print("Kurtosis: %f" % abs(data).kurt())
    print("Shapiro Test: %f" % shap_t)
    print("Shapiro p_value: %f" % shap_p)
    
    if shap_p > 0.05:
        print('The distribution is normal')
    else:
        print('The distribution is not normal')
        
    
check_normality(df['age'],'Age')
print('-------------------------')
check_normality(df['creatinine_phosphokinase'],'Creatinine Phosphokinase')
print('-------------------------')
check_normality(df['ejection_fraction'],'Ejection Fraction')
print('-------------------------')
check_normality(df['platelets'],'Platelets')
print('-------------------------')
check_normality(df['serum_creatinine'],'Serum Creatinine')
print('-------------------------')
check_normality(df['serum_sodium'],'Serum Sodium')
print('-------------------------')
check_normality(df['time'],'Time')

None of our feature is normally distributed but here we are going to use Random Forest Classifier which is robust to outliers..

Here we have used Gussian Distribution to overcome from outliers effect as age column is nearly normally distributed

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.boxplot(data=df, width= 0.5,ax=ax,  fliersize=3)
plt.xticks(rotation = 45)

In [ ]:



uppper_boundary=df['platelets'].mean() + 3* df['platelets'].std()
lower_boundary=df['platelets'].mean() - 3* df['platelets'].std()
print(lower_boundary), print(uppper_boundary),print(df['platelets'].mean())

In [ ]:
data=df.copy()

In [ ]:
data.loc[data['platelets']>=556770,'platelets']=556770

data.loc[data['platelets']<=-30054,'platelets']=-30054


In [ ]:
data

In [ ]:
data.boxplot(column="platelets")

## EDA of continious feature and dependent variable

In [ ]:
g = sns.FacetGrid(data, col='DEATH_EVENT', size=5)
g.map(plt.hist, 'age', bins=10)

In [ ]:
data['AgeBand'] = pd.cut(data['age'], 4)
df_age=data[['AgeBand', 'DEATH_EVENT']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)
df_age

In [ ]:
g = sns.factorplot(x="AgeBand",y="DEATH_EVENT",data=df_age,kind="bar", size = 6 , 
palette = "muted")
g = g.set_ylabels("Death Events probablity")

We can clearly oberserve that the person who have age between 40-54 have least chanse of getting heart attack and person who is above 80 have higher chanse of getting heart attack

In [ ]:
g = sns.FacetGrid(data, col='DEATH_EVENT', size=5)
g.map(plt.hist, 'platelets', bins=10)

In [ ]:
data['plateletsBand'] = pd.cut(data['platelets'], 3)
df_platelets=data[['plateletsBand', 'DEATH_EVENT']].groupby(['plateletsBand'], as_index=False).mean().sort_values(by='plateletsBand', ascending=True)
df_platelets

In [ ]:
g = sns.factorplot(x="plateletsBand",y="DEATH_EVENT",data=df_platelets,kind="bar", size = 8 , 
palette = "muted")
g = g.set_ylabels("Death Events probablity")


In [ ]:
g = sns.FacetGrid(data, col='DEATH_EVENT', size=5)
g.map(plt.hist, 'creatinine_phosphokinase', bins=10)

In [ ]:
data['creatinine_phosphokinaseBand'] = pd.cut(data['creatinine_phosphokinase'], [0,1000,8000])
df_creatinine_phosphokinase=data[['creatinine_phosphokinaseBand', 'DEATH_EVENT']].groupby(['creatinine_phosphokinaseBand'], as_index=False).mean().sort_values(by='creatinine_phosphokinaseBand', ascending=True)
df_creatinine_phosphokinase

In [ ]:
g = sns.factorplot(x="creatinine_phosphokinaseBand",y="DEATH_EVENT",data=df_creatinine_phosphokinase,kind="bar", size = 8 , 
palette = "muted")
g = g.set_ylabels("Death Events probablity")


In [ ]:
data['ejection_fractionBand'] = pd.cut(data['ejection_fraction'], 2)
df_ejection_fraction=data[['ejection_fractionBand', 'DEATH_EVENT']].groupby(['ejection_fractionBand'], as_index=False).mean().sort_values(by='ejection_fractionBand', ascending=True)
df_ejection_fraction

In [ ]:
g = sns.factorplot(x="ejection_fractionBand",y="DEATH_EVENT",data=df_ejection_fraction,kind="bar", size = 8 , 
palette = "muted")
g = g.set_ylabels("Death Events probablity")


In [ ]:
g = sns.FacetGrid(data, col='DEATH_EVENT', size=5)
g.map(plt.hist, 'serum_creatinine', bins=10)

In [ ]:
data['serum_creatinineBand'] = pd.cut(data['serum_creatinine'], 3)
df_serum_creatinine=data[['serum_creatinineBand', 'DEATH_EVENT']].groupby(['serum_creatinineBand'], as_index=False).mean().sort_values(by='serum_creatinineBand', ascending=True)
df_serum_creatinine

In [ ]:
g = sns.factorplot(x="serum_creatinineBand",y="DEATH_EVENT",data=df_serum_creatinine,kind="bar", size = 8 , 
palette = "muted")
g = g.set_ylabels("Death Events probablity")


In [ ]:
g = sns.FacetGrid(data, col='DEATH_EVENT', size=5)
g.map(plt.hist, 'serum_sodium', bins=10)

In [ ]:
data['serum_sodiumBand'] = pd.cut(data['serum_sodium'], 3)
df_serum_sodium=data[['serum_sodiumBand', 'DEATH_EVENT']].groupby(['serum_sodiumBand'], as_index=False).mean().sort_values(by='serum_sodiumBand', ascending=True)
df_serum_sodium

In [ ]:
g = sns.factorplot(x="serum_sodiumBand",y="DEATH_EVENT",data=df_serum_sodium,kind="bar", size = 8 , 
palette = "muted")
g = g.set_ylabels("Death Events probablity")


In [ ]:
g = sns.FacetGrid(data, col='DEATH_EVENT', size=5)
g.map(plt.hist, 'time', bins=10)

In [ ]:
data['timeBand'] = pd.cut(data['time'], 2)
df_time=data[['timeBand', 'DEATH_EVENT']].groupby(['timeBand'], as_index=False).mean().sort_values(by='timeBand', ascending=True)
df_time

In [ ]:
g = sns.factorplot(x="timeBand",y="DEATH_EVENT",data=df_time,kind="bar", size = 8 , 
palette = "muted")
g = g.set_ylabels("Death Events probablity")


In [ ]:
data.drop(['plateletsBand','creatinine_phosphokinaseBand','ejection_fractionBand'], axis=1, inplace=True)

In [ ]:
data

Here we have decided to change age column into categorical feature

In [ ]:
#for age
data.loc[ data['age'] <= 53.75, 'age'] = 0
data.loc[(data['age'] > 53.75) & (data['age'] <= 67.5), 'age'] = 1
data.loc[(data['age'] > 67.5) & (data['age'] <= 81.25), 'age'] = 2
data.loc[ data['age'] > 81.25, 'age'] = 3












In [ ]:
data

In [ ]:
#for serum_creatinine

#data.loc[ data['serum_creatinine'] <= 3.467, 'serum_creatinine'] = 0
#data.loc[(data['serum_creatinine'] > 3.467) & (data['serum_creatinine'] <= 6.433), 'serum_creatinine'] = 1
#data.loc[ data['serum_creatinine'] > 6.433, 'serum_creatinine'] = 2


In [ ]:
#for serum_sodium

#data.loc[ data['serum_sodium'] <= 124.667, 'serum_sodium'] = 0
#data.loc[(data['serum_sodium'] > 124.667) & (data['serum_sodium'] <= 136.333), 'serum_sodium'] = 1
#data.loc[ data['serum_sodium'] > 136.333, 'serum_sodium'] = 2



In [ ]:
#for time

#data.loc[ data['time'] <= 144.5, 'time'] = 0
#data.loc[(data['time'] > 97.667) & (data['time'] <= 191.333), 'time'] = 1
#data.loc[ data['time'] > 144.5, 'time'] = 1


#### I tried to convert 4 continious feature into caterogorical variable but my model gets overfitted and it only worked with Age column if you know why it happend then plz comment down

In [ ]:
data


In [ ]:
data.drop(['AgeBand','serum_creatinineBand','serum_sodiumBand','timeBand'],axis=1, inplace=True)

In [ ]:
data

In [ ]:
df_New=pd.get_dummies(data,columns=['anaemia','diabetes','high_blood_pressure','sex','smoking'],drop_first=True)
df_New.head()

In [ ]:
#correlation map
df_New.corr
f,ax = plt.subplots(figsize=(30,30))
sns.heatmap(df_New.corr(), annot =True, linewidth =".5", fmt =".2f", cmap='cividis')
plt.show()

There is no high correlation has found between categories so we dont need to drop any column

## Random Forest Classifier

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score

from IPython.display import Image  
from sklearn.tree import export_graphviz


In [ ]:
X = data.drop(columns = ['DEATH_EVENT'])
y = data['DEATH_EVENT']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.30, random_state= 355)

In [ ]:
clf = DecisionTreeClassifier( min_samples_split= 2)
clf.fit(x_train,y_train)

In [ ]:
clf.score(x_test,y_test)

In [ ]:
clf2 = DecisionTreeClassifier(criterion = 'entropy', max_depth =24, min_samples_leaf= 1)
clf2.fit(x_train,y_train)

In [ ]:
clf2.score(x_test,y_test)

In [ ]:
rand_clf = RandomForestClassifier(criterion= 'entropy',
 max_depth = 15,
 max_features = 'log2',
 min_samples_leaf =1,
 min_samples_split= 8,
 n_estimators = 90,random_state=42,ccp_alpha=0.0,bootstrap=True)

In [ ]:
rand_clf.fit(x_train,y_train)

In [ ]:
rand_clf.score(x_train,y_train)

In [ ]:
rand_clf.score(x_test,y_test)

In [ ]:
grid_param = {
    "n_estimators" : [90,100,115],
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf' : [1,2,3,4,5],
    'min_samples_split': [4,5,6,7,8],
    'max_features' : ['auto','log2']
    
    
    

    
    
}

In [ ]:
grid_search = GridSearchCV(estimator=rand_clf,param_grid=grid_param,cv=5,n_jobs =-1,verbose = 3)

In [ ]:
rand_clf = RandomForestClassifier(criterion= 'gini',max_features= 'auto',min_samples_leaf= 3,min_samples_split= 7,n_estimators= 115,random_state=6)

In [ ]:
rand_clf.fit(x_train,y_train)

In [ ]:
rand_clf.score(x_train,y_train)

In [ ]:
rand_clf.score(x_test,y_test)

In [ ]:
#Calculating Prediction
y_pred = rand_clf.predict(x_test)
y_pred_prob = rand_clf.predict_proba(x_test)

In [ ]:
# Confusion Matrix
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat

In [ ]:
true_positive = conf_mat[0][0]
false_positive = conf_mat[0][1]
false_negative = conf_mat[1][0]
true_negative = conf_mat[1][1]

In [ ]:
# Recall
Recall = true_positive/(true_positive+false_negative)
Recall

In [ ]:
# Precison
Precision = true_positive/(true_positive+false_positive)
Precision

In [ ]:
# F1 Score
F1_Score = 2*(Recall * Precision) / (Recall + Precision)
F1_Score

In [ ]:
# Area Under Curve
auc = roc_auc_score(y_test, y_pred)
auc

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#Calculating Confusion Matrix
CM = confusion_matrix(y_test, y_pred)
print('Confusion Matrix is : \n', CM)

# drawing confusion matrix
sns.heatmap(CM, center = True,cmap='YlGnBu')
plt.show()

In [ ]:
#Calculating Details
print('RandomForestClassifierModel Train Score is : ' , rand_clf.score(x_train, y_train))
print('RandomForestClassifierModel Test Score is : ' , rand_clf.score(x_test, y_test))
print('RandomForestClassifierModel precision Score is : ' , Precision)
print('RandomForestClassifierModel Recall Score is : ' , Recall)
print('RandomForestClassifierModel F1 Score is : ' , F1_Score)
print('RandomForestClassifierModel auc Score is : ' , auc)
      
